# Linear Regression Notebook

In this notebook I will train a very simple model end to end.

### Description

This is the May 2025 calories prediction competition.

### Files
1. train.csv
2. test.csv
3. sample_submission.csv

### Evaluation

The evaluation metric is the RMSLE.

Submission File
For each id in the test set, you must predict the number of minutes listened. The file should contain a header and have the following format:

- id,Listening_Time_minutes
- 26570,0.2
- 26571,0.1
- 26572,0.9
- etc.

## Package Importing

In [1]:
# general python libraries
import numpy as np

# dataframe and data manipulation library
import pandas as pd

# machine learning libraries
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import root_mean_squared_log_error
import optuna

from scripts.data_processing import preprocessing, postprocessing

## Data Importing

In [2]:
TARGET_COLUMN = 'Calories'

In [3]:
folder_path = '../data/raw'
df_train = pd.read_csv(f'{folder_path}/train.csv', index_col='id')
df_test = pd.read_csv(f'{folder_path}/test.csv', index_col='id')
df_sample_submission = pd.read_csv(f'{folder_path}/sample_submission.csv',index_col='id')

## Data Cleaning

In [ ]:
X_train = df_train.loc[:,df_train.columns != TARGET_COLUMN]
y_train = df_train.loc[:,TARGET_COLUMN]

X_train = preprocessing(X_train)
X_train = postprocessing(X_train)

## Model fitting

### Train Test Split

Splitting data into groupings for model fitting

In [101]:
from optuna.integration import LightGBMPruningCallback

In [102]:
def get_rmsle(preds,eval_data):

    y_preds_negatives_removed = np.clip(preds, a_min= 0, a_max = np.inf)

    rmsle = root_mean_squared_log_error(y_preds_negatives_removed, eval_data)

    # Metric must have the following format to be accepted
    # eval_name, eval_result, is_higher_better
    return 'rmsle', rmsle, False

In [ ]:
from xgboost.callback import EarlyStopping
from optuna.integration import LightGBMPruningCallback

def objective(trial: optuna.Trial, X = X_train, y = y_train) -> int:

    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2,random_state=42)

    param = {
        'lambda_l1': trial.suggest_float('lambda_l1', 0.001, 0.002, log=True),
        'lambda_l2': trial.suggest_int('lambda_l2', 6, 12),
        "bagging_freq":  trial.suggest_int('bagging_freq', 3, 5),
        "bagging_fraction": trial.suggest_float('bagging_fraction', 0.56, 0.58),
        "feature_fraction": trial.suggest_float('feature_fraction', 0.75, 0.76),
        'learning_rate': trial.suggest_float('learning_rate', 0.04, 0.12),
        "max_depth":  trial.suggest_int('max_depth', 3, 6),  

        #fixed part       
        'random_state': 64, 
        'num_leaves': 64,        
        'objective': 'regression_l1',
        'reg_lambda': 1.,
        'reg_alpha': .1,
        'n_estimators': 100, 
        'colsample_bytree': .5, 
        'min_child_samples': 10,
        'subsample_freq': 3,
        'subsample': .9,
        'importance_type': 'gain', 
        'verbosity':0
    }

    callbacks = [
        LightGBMPruningCallback(
            trial=trial,
            metric='rmsle',
            valid_name="valid_0"
        ),
        lgb.callback.early_stopping(stopping_rounds=50,min_delta=0.001)
    ]

    model = lgb.LGBMRegressor(
        **param,
        force_row_wise=True,
    )  


    model.fit(
        train_x, 
        train_y,
        eval_set=[(test_x, test_y)],
        eval_metric=get_rmsle,
        callbacks=callbacks,
    )

    
    preds = model.predict(test_x)

    eval_name, rmsle, higher_is_better = get_rmsle(eval_data=test_y,preds=preds)

    return rmsle

In [172]:
pruner = optuna.pruners.PercentilePruner(
    percentile=30.0,
    n_startup_trials=5,
    n_warmup_steps=800,
    interval_steps=500,
    n_min_trials=5
)
study = optuna.create_study(direction="minimize", pruner=pruner)

[I 2025-05-05 02:08:54,794] A new study created in memory with name: no-name-1433ef58-7d20-450d-be3f-2f288580e30a


In [ ]:
study.optimize(
    objective, 
    n_trials=300, 
    show_progress_bar=True
)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

  0%|          | 0/50 [00:00<?, ?it/s]

/var/folders/ph/4n426_w151s3mcqb32z08qq00000gn/T/ipykernel_97034/804059996.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.



[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=3 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l1 is set=0.001046228954058943, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.001046228954058943
[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=1.0 will be ignored. Current value: lambda_l2=8
[LightGBM] [Warning] bagging_fraction is set=0.5777014021861073, subsample=0.9 will be ignored. Current value: bagging_fraction=0.5777014021861073
[LightGBM] [Warning] feature_fraction is set=0.7546078620086498, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.7546078620086498
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=3 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l1 is set=0.001046228954058943, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.001046228954058943
[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=1.0 will be ignored. Current value: lambda_l2=8
[LightGBM

/var/folders/ph/4n426_w151s3mcqb32z08qq00000gn/T/ipykernel_97034/804059996.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.



[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=3 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l1 is set=0.0011040705407441974, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.0011040705407441974
[LightGBM] [Warning] lambda_l2 is set=7, reg_lambda=1.0 will be ignored. Current value: lambda_l2=7
[LightGBM] [Warning] bagging_fraction is set=0.5661026936922932, subsample=0.9 will be ignored. Current value: bagging_fraction=0.5661026936922932
[LightGBM] [Warning] feature_fraction is set=0.752184288191593, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.752184288191593
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=3 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l1 is set=0.0011040705407441974, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.0011040705407441974
[LightGBM] [Warning] lambda_l2 is set=7, reg_lambda=1.0 will be ignored. Current value: lambda_l2=7
[LightG

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/var/folders/ph/4n426_w151s3mcqb32z08qq00000gn/T/ipykernel_97034/804059996.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/var/folders/ph/4n426_w151s3mcqb32z08qq00000gn/T/ipykernel_97034/804059996.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [170]:
study.best_params

{'lambda_l1': 0.0019308425132961418,
 'lambda_l2': 8.612376766701301,
 'bagging_freq': 5,
 'bagging_fraction': 0.5799027628764685,
 'feature_fraction': 0.7594543478006752,
 'learning_rate': 0.11960025124188359,
 'verbosity': 8,
 'max_depth': 6}

In [ ]:
{'lambda_l1': 0.0014667690638638649, 
 'lambda_l2': 7.251502224420871, 
 'bagging_freq': 4, 
 'bagging_fraction': 0.5655411849965026, 
 'feature_fraction': 0.7526024056840769, 
 'learning_rate': 0.11861277077083375, 
 'verbosity': 7, 
 'max_depth': 6}

In [175]:
study.best_trial

FrozenTrial(number=21, state=1, values=[0.06454755532798163], datetime_start=datetime.datetime(2025, 5, 5, 2, 9, 27, 441641), datetime_complete=datetime.datetime(2025, 5, 5, 2, 9, 29, 239467), params={'lambda_l1': 0.0014503954247443312, 'lambda_l2': 9, 'bagging_freq': 4, 'bagging_fraction': 0.5607354947778781, 'feature_fraction': 0.7572719060238033, 'learning_rate': 0.11912190285130149, 'max_depth': 6}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.9198769637079545, 1: 0.8615169293712345, 2: 0.8089540522610478, 3: 0.7655248875136389, 4: 0.7212367749146503, 5: 0.6822877148543801, 6: 0.6450335267596733, 7: 0.6119246957456035, 8: 0.5760927597852809, 9: 0.5449705698921407, 10: 0.514766668232896, 11: 0.48686437689456885, 12: 0.45895337568173733, 13: 0.4328239230233015, 14: 0.40658848339783277, 15: 0.38423165000641, 16: 0.36250745200217044, 17: 0.3420719370185128, 18: 0.32258369164919215, 19: 0.3047496335408999, 20: 0.2864328896583601, 21: 0.26948295014757045, 22: 0.2542414188396

# Optuna Visualisations

In [162]:
optuna.visualization.plot_optimization_history(study)

In [163]:
optuna.visualization.plot_param_importances(study)

In [164]:
optuna.visualization.plot_slice(study)

In [165]:
optuna.visualization.plot_contour(study)

In [166]:
optuna.visualization.plot_intermediate_values(study)